In [ ]:
#The first step is to install all the required libraries
!pip install  beautifulsoup4 transformers requests

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#In this Step We will import the models used inside this project from abovee installed  libraries 
#when we pass string to this transformer tokennizer it help us to convert this into a sequence of  numbers That we can than pass to NLP model
from transformers import AutoTokenizer
#it gives us the architecture that we are going to load into NLP model
from transformers import AutoModelForSequenceClassification
#In this we will use    function to extract the high no. result
import torch
#The request is used to grab the data 
import requests
#This allows us to extract the data that we need
from bs4 import BeautifulSoup
#it is use to extract the specific comments that we want
import re

In [ ]:
#In This we are impoting the pretrained model That we will Be using 
tokenizer= AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
#Now we are downloading this pretrained model and storing in object 'model'
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

In [ ]:
#Now we Are testing how a token convert a string into a sequence of numbers
#tokens=tokenizer.encode( 'I Completly hate you,You are worst of all',return_tensors='pt') #bad mood
tokens=tokenizer.encode('I Completly love this , you are best of all',return_tensors='pt') #Good mood

In [ ]:
#these are the tokens that is our string converted into sequence
tokens[0]

tensor([  101,   151, 41495, 10563, 11157, 10372,   117, 10855, 10320, 11146,
        10108, 10367,   102])

In [ ]:
#In this we have converted the tokens back into string
tokenizer.decode(tokens[0])

'[CLS] i completly love this, you are best of all [SEP]'

In [ ]:
#We Are passing the token in above defined model and storing it in 'result' 
result=model(tokens)
result

SequenceClassifierOutput([('logits',
                           tensor([[-2.4914, -2.6640, -1.0223,  1.3361,  3.9261]],
                                  grad_fn=<AddmmBackward0>))])

In [ ]:
#This Shows The highest value result. And the highest value represent the sentiment
torch.argmax(result.logits)
#the score is from 1 to 5 that represents from our bad to great mood it can be find out by following commmand
int(torch.argmax(result.logits))+1

5

In [ ]:
#Extracting the reviews from a website using beautifulsoup
#Will be explained by Joshiii
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [ ]:
#this shoows respose to our request [200] request is successfully accepted and given some output
r

<Response [200]>

In [ ]:
#This shows what text we have got as result
r.text

'<!DOCTYPE html><html lang="en-US" prefix="og: http://ogp.me/ns#" style="margin: 0;padding: 0; border: 0; font-size: 100%; font: inherit; vertical-align: baseline;"><head><script>document.documentElement.className=document.documentElement.className.replace(/\x08no-js\x08/,"js");</script><meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /><meta http-equiv="Content-Language" content="en-US" /><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"><link rel="mask-icon" sizes="any" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/b2bb2fb0ec9c/assets/img/logos/yelp_burst.svg" content="#FF1A1A"><link rel="shortcut icon" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/b05852393ae5/assets/img/logos/favicon.ico"><script> window.ga=window.ga||function(){(ga.q=ga.q||[]).push(arguments)};ga.l=+new Date;window.ygaPageStartTime=new Date().getTime();</script><script async src="https://www.google-analytics.com/ana

In [ ]:
#Load riviews into dataframe and score
#in this we are creating a column name 'review' and storing our data frame into in
df = pd.DataFrame(np.array(reviews), columns=['review'])
#let see how our df looks like(lets see only head part)
df.head()

,review
0,Great coffee and vibe. That's all you need. C...
1,Great coffee and vibe. That's all you need. C...
2,I came to Social brew cafe for brunch while ex...
3,Ricotta hot cakes! These were so yummy. I ate ...
4,I went here a little while ago- a beautiful mo...


In [ ]:
df['review'].iloc[3]

"Ricotta hot cakes! These were so yummy. I ate them pretty fast and didn't share with anyone because they were that good ;). I ordered a green smoothie to balance it all out. Smoothie was a nice way to end my brekkie at this restaurant. Others with me ordered the salmon Benedict and the smoked salmon flatbread. They were all delicious and all plates were empty. Cheers!"

In [ ]:
#Now we will Create a function that will take Reviews and  give us output as no. output
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
#let see what is the review of customers
sentiment_score(df['review'].iloc[1])

4

In [ ]:
#the above shows sentimantal analysis now if we have to do analysis for all the reviews we will be using lamda function
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512])) #we have use 512 because NLP Model that we use has limitation that it can take only up to 512 token as embeddings
#df['review'] #this we will show us all the review present in the review column

In [ ]:
df

,review,sentiment
0,Great coffee and vibe. That's all you need. C...,5
1,Great coffee and vibe. That's all you need. C...,4
2,I came to Social brew cafe for brunch while ex...,5
3,Ricotta hot cakes! These were so yummy. I ate ...,5
4,I went here a little while ago- a beautiful mo...,2
5,We came for brunch twice in our week-long visi...,4
6,Ron & Jo are on the go down under and Wow! We...,5
7,Good coffee and toasts. Straight up and down -...,5
8,This place is a gem. The ambiance is to die fo...,3
9,Delicious. The waitress was hot. The burger wa...,4


In [ ]:
#df['review'].iloc[3]